In [1]:
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import requests
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [2]:
!unzip /content/SPIQA_testA_Images.zip

Archive:  /content/SPIQA_testA_Images.zip
   creating: SPIQA_testA_Images/
   creating: SPIQA_testA_Images/1809.03449v3/
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Table3-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Table1-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure3-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure1-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure4-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure2-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Table2-1.png  
   creating: SPIQA_testA_Images/1710.01507v4/
  inflating: SPIQA_testA_Images/1710.01507v4/1710.01507v4-Table1-1.png  
  inflating: SPIQA_testA_Images/1710.01507v4/1710.01507v4-Figure1-1.png  
   creating: SPIQA_testA_Images/1709.02755v5/
  inflating: SPIQA_testA_Images/1709.02755v5/1709.02755v5-Table1-1.png  
  inflating: SPIQA_testA_Images/1709.02755v5/1709.02

In [3]:
df = pd.read_csv('/content/sample.csv')
df

,paper,question,answer,reference_figure,reference_figure_caption,generated_answer
0,1611.04684v1,What are the main differences between the educ...,The Bonaparte school focuses on outdoor physic...,1611.04684v1-Table1-1.png,A difficult example from QA,NaN
1,1611.04684v1,Which model performs the best for response sel...,The KEHNN model performs the best for response...,1611.04684v1-Table4-1.png,Table 4: Evaluation results on response selection,NaN
2,1611.04684v1,Which model performs best on the Ubuntu datase...,KEHNN,1611.04684v1-Table5-1.png,Accuracy on different length of text,NaN
3,1611.04684v1,What is the role of the knowledge gates in the...,The knowledge gates are responsible for select...,1611.04684v1-Figure1-1.png,Architecture of KEHNN,NaN
4,1611.04684v1,How does the average number of answers per que...,The training set has a higher average number o...,1611.04684v1-Table2-1.png,Table 2: Statistics of the answer selection da...,NaN
...,...,...,...,...,...,...
661,1809.04276v2,How does the discriminator in the proposed REA...,The discriminator takes as input a response an...,1809.04276v2-Figure1-1.png,Figure 1: An overview of our proposed approach...,NaN
662,1710.01507v4,What is the role of the LSTM network in the mo...,The LSTM network is used to process the post t...,1710.01507v4-Figure1-1.png,Model Architecture,NaN
663,1709.00139v4,Which method generally achieved a lower object...,"For all datasets presented, Incremental SVM ac...",1709.00139v4-Table1-1.png,Table 1: Experimental Results of FISVDD and In...,NaN
664,1809.01989v2,Which method achieved the highest tracking acc...,The Ridge method achieved the lowest sum of ab...,1809.01989v2-Table1-1.png,Table 1. Absolute percentage errors for differ...,NaN


In [4]:
def load_image_and_question(image_path):
    path = "/content/SPIQA_testA_Images/" + image_path
    raw_image = Image.open(path).convert('RGB')
    display(raw_image)
    return raw_image

In [ ]:
answers = []
for i, row in df.iterrows():
    image_path = row['paper'] + '/' + row['reference_figure']
    # question = "Given: " + row['reference_figure_caption'].strip() + '\n' + row['question'].strip()
    question = row['question'].strip()
    image = load_image_and_question(image_path)
    print("Question:", question)
    print("Actual Answer:", row['answer'].strip())
    inputs = processor(image, question, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(**inputs)
        answer = processor.decode(output[0], skip_special_tokens=True)
        answers.append(answer)
    print("Answer:", answer)
    print()

In [6]:
df['generated_answer'] = answers
df.to_csv("blip_results_2.csv", index=False)